In [1]:
from neural_mfg import dgm_net
import numpy as np
import tensorflow as tf

nn = dgm_net()

## Warmstart
We preprocess the NNs to display the initial value $m_0$ everywhere in the domain.

In [2]:
# for this we have
# "N_b" : 400,
# "N_in": 100000

nn.learning_rate = 0.0001
nn.warmstart_simple()
nn.draw()
total_mass = (tf.reduce_mean(nn.phi_theta(nn.all_pts)*nn.gamma_theta(nn.all_pts))*(2*nn.lx)*(2*nn.ly)-nn.total_mass)**2
print(total_mass)

WS step     0, loss phi= 3.424e-01, loss gamma= 1.374e+00


KeyboardInterrupt: 

## Transfer learning w.r.t. the potential V

Starting from $V=0$, where the warmstart should be a solution (i.e. without the obstacle we are just solving a heat equation), we increase the value of the potential progressively, attributing more and more importance to the presence of the obstacle.

In [ ]:
nn.training_steps = 50
nn.learning_rate = 0.0001
    
potentials = np.linspace(0,101,100)
for potential in potentials:
    print("V = -"+str(potential))
    nn.pot = -potential
    nn.train()

In [ ]:
# learning with resampling at every iteration

# reducing the point pool
# "N_b" : 200,
# "N_in": 1000
nn.N_b = 200
nn.N_in = 1000
nn.X_b, nn.X_in, nn.X_out = nn.sample_room()
nn.all_pts   = tf.constant(tf.concat([nn.X_out,nn.X_in,nn.X_b],axis = 0))

nn.pot = -100
nn.weight_HJB = 100 # this is a new attribute of the DGM class, it allowes to penalize more the 
                    # residual of the HJB equation (default is 1). This is why you see such a chunky
                    # error in res_HJB: it's inflated by this weight.
nn.M = 50
nn.resampling_step = 10 # alternative to sample every step
nn.training_steps = 1000
nn.learning_rate = 0.0001
nn.train(resampling = True)

# run many times until desired convergence


    #iter          res_HJB          res_KFP          res_b           total_mass      |   Loss_total
----------------------------------------------------------------------------------------------------
        10         1.141e+02        5.781e+01        1.486e+00         2.423e+03       |   1.390e+04
        20         1.096e+02        1.367e+02        7.906e-02         3.641e+02       |   1.146e+04
        30         1.054e+02        1.475e+02        2.376e-01         1.657e+01       |   1.070e+04
        40         1.015e+02        1.111e+02        3.069e-01         1.685e+01       |   1.028e+04
    

## Plotting and saving the results

In [ ]:
nn.save()

## Loading network after saving

In [ ]:
from neural_mfg import dgm_net 
import numpy as np
import tensorflow as tf
directory_name = "trainings/November 28, 2023 19-21-29" # this is an example working on my local machine
nn = dgm_net(directory_name)
nn.load(directory_name)
# now you are ready to keep on with the training
# to do: add saving and loading for the loss components